<a href="https://colab.research.google.com/github/gemepolimi/gemepolimi/blob/main/NOAA_DATA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import xarray as xr
import pandas as pd
import numpy as np

# Install the xlsxwriter package if it's not already installed
try:
    import xlsxwriter
except ImportError:
    !pip install xlsxwriter
    import xlsxwriter

# 1. Data Extraction & Conversion
ds = xr.open_dataset('subset.nc')
# Standard Addis Ababa Coordinates
aa = ds['prate'].sel(lat=9.03, lon=38.74, method='nearest') * 86400

df = aa.to_dataframe(name='Rain').reset_index()
df['Year'], df['Month'], df['Day'] = df.time.dt.year, df.time.dt.month, df.time.dt.day
m_labels = ['J', 'F', 'M', 'A', 'M', 'J', 'J', 'A', 'S', 'O', 'N', 'D']

# 2. Setup Excel Writer
file_name = 'ADDIS_ABEBA_NOAA_FORMATTED.xlsx'
writer = pd.ExcelWriter(file_name, engine='xlsxwriter')
workbook = writer.book

# --- DEFINE STYLES ---
# Data cells: 2 decimals, commas, centered
cell_style = workbook.add_format({'num_format': '#,##0.00', 'align': 'center', 'border': 1})
# Header: Bold, Green background
header_style = workbook.add_format({'bold': True, 'align': 'center', 'bg_color': '#C6E0B4', 'border': 1})
# Totals Row: Bold, Blue background
total_row_style = workbook.add_format({'bold': True, 'num_format': '#,##0.00', 'bg_color': '#DDEBF7', 'border': 1, 'align': 'center'})
# Annual/Summary Style: Bold, Orange background
summary_style = workbook.add_format({'bold': True, 'num_format': '#,##0.00', 'bg_color': '#FCE4D6', 'border': 1, 'align': 'center'})

# 3. Process Daily Sheets
for year, group in df.groupby('Year'):
    sheet_data = group.pivot(index='Day', columns='Month', values='Rain')
    sheet_data = sheet_data.reindex(index=range(1, 32), columns=range(1, 13))
    sheet_data.columns = m_labels

    # Calculate Totals
    sheet_data.loc['Monthly Total'] = sheet_data.sum(axis=0)
    sheet_data['Total Annual'] = np.nan
    sheet_data.at['Monthly Total', 'Total Annual'] = sheet_data.loc['Monthly Total'].sum()

    sheet_name = str(int(year))
    sheet_data.to_excel(writer, sheet_name=sheet_name)
    worksheet = writer.sheets[sheet_name]

    # Apply Formatting
    worksheet.set_column('A:M', 10)
    worksheet.set_column('N:N', 15) # Wider column for Annual Total

    # Format Headers (A1:N1)
    for col_num, value in enumerate(['Day'] + list(sheet_data.columns)):
        worksheet.write(0, col_num, value, header_style)

    # Format Body (Rainfall data)
    worksheet.conditional_format('B2:M32', {'type': 'no_errors', 'format': cell_style})

    # Format Monthly Total Row
    worksheet.set_row(32, None, total_row_style)

    # Format Total Annual Column and Cell
    worksheet.conditional_format('N2:N32', {'type': 'no_errors', 'format': summary_style})
    worksheet.write('N33', sheet_data.at['Monthly Total', 'Total Annual'], summary_style)

# 4. Process Annual Summary Sheet
monthly_sum = df.groupby(['Year', 'Month'])['Rain'].sum().reset_index()
summary = monthly_sum.pivot(index='Year', columns='Month', values='Rain')
summary.columns = m_labels
summary['Total Annual'] = summary.sum(axis=1)
summary.loc['Media General'] = summary.mean(axis=0)

summary.to_excel(writer, sheet_name='Annual_Summary')
summary_ws = writer.sheets['Annual_Summary']
summary_ws.set_column('A:N', 12)

# Format the Summary Headers
for col_num, value in enumerate(['Year'] + m_labels + ['Total Annual']):
    summary_ws.write(0, col_num, value, header_style)

# Format the body and the "Media General" row
summary_ws.conditional_format('B2:N1000', {'type': 'no_errors', 'format': cell_style})
last_row = len(summary)
summary_ws.set_row(last_row, None, summary_style)

writer.close()

print(f"Excel styling complete! File saved as: {file_name}")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.3/175.3 kB 6.7 MB/s eta 0:00:00
Excel styling complete! File saved as: ADDIS_ABEBA_NOAA_FORMATTED.xlsx
